# Eliminar Efecto de histéresis
Dalia Camacho

El propósito de esta tarea es modificar el MaxSelector para eliminar el efecto de histéresis. Para lograr esto creamos un MaxSelector con tres capas de neuronas. Para un input de $n$ estímulos, se tiene una primera capa $u^{(1)}$ (con $n$ neuronas), tal que el valor en cada neurona $u_i^{(1)}$ es el valor del estímulo externo $s_i$ más el valor de la neurona $v$.
\begin{equation}
    u_i^{(1)}=s_i+v
\end{equation}

En la segunda capa $u^{(2)}$ (con $n$ neuronas) se realiza el cálculo $u_i^{(2)}=f_3(u^{(u)}_i)$ para cada $i \in \{1\dots n\}$. Se tiene que

\begin{equation}
u_i^{(2)}= f_3(u^{(u)}_i)= f_2(u_i)-\frac{\sum_{j\neq i} f_2(u^{(1)}_j)}{\sum_{j\neq i}f_1(u^{(1)}_j)},
\end{equation}

donde

\begin{equation}
f_1(x)=
\begin{cases} 
      0 & x\leq 0 \\
     1 &  e.o.c.
   \end{cases}
\end{equation} 

y 

\begin{equation}
f_2(x)=
\begin{cases} 
      0 & x\leq 0 \\
     x &  e.o.c. 
   \end{cases}.
\end{equation} 

En la neurona $v$ se realiza el cálculo $v=v+f_4(u^{(2)})$ donde $f_4$ está dada por:

\begin{equation}
f_4(u)=\begin{cases}
0 & \sum_{i=1}^n f_1(u) = 1\\
-\frac{\sum_{i=1}^n f_1(u)}{n} & e.o.c
\end{cases}.
\end{equation}

Por último el máximo está indicado por $f_1(u^{(2)}$.

Cargamos la librería numpy.

In [1]:
# Importar librerías numpy
import numpy as np

Definimos la función MaxSelector3capas así como las funciones auxiliares $f_i$ con $i \in \{1,2,3,4\}$.

In [2]:
# Definimos f1
def f1(vec):
    return np.where(vec<=0,0.0,1.0)

# Definimos f2
def f2(vec):
    return np.where(vec<=0,0,vec)

# Definimos f3
def f3(vec):
    n     = np.size(vec)
    sol   = np.zeros((n,1)) 
    f1vec = f1(vec)
    f2vec = f2(vec)
    for i in range(0,(n)):
        sol[i] = f2vec[i] -(np.sum(f2vec)-f2vec[i])/(np.sum(f1vec)-f1vec[i])
    return sol

# Definimos f4
def f4(vec):
    if(np.sum(f1(vec))==1):
        sol = 0
    else:
        sol = -np.sum(f1(vec))/np.size(vec)
    return sol

# Definimos la función que describe el comportamiento de las neuronas    
def MaxSelector3capas(stimulus, time, delta):
    n  = np.size(stimulus)
    v  = np.zeros((n,1)) 
    v0 = v
    for t in np.arange(0.0, time, delta):
        u1 = stimulus + v
        u2 = f3(u1)
        v  = v+f4(u2)
        
    return f1(u2).T

Probamos el MaxSelector con tres capas para un estímulo inicial.

In [3]:
# Definir parámetros del sistema a excepción del estímulo
time  = 10
delta = 0.05

In [4]:
# Definir estímulo y probar MaxSelector
stimulus =np.array([[0.0, 1.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).T
MaxSelector3capas(stimulus, time, delta)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
# Definir estímulo y probar MaxSelector
stimulus =np.array([[0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.22, 0.0]]).T
MaxSelector3capas(stimulus, time, delta)

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

Ahora modificamos el código para incluir la posibilidad de un segundo estímulo. Para esto se agrega un lag que es el tiempo en el que ocurre en el segundo estímulo, considerando que el primer estímulo ocurre al tiempo $0$. Y se añade un segundo estímulo en las variables de entrada con lo que ahora tenemos:
\begin{equation}
    u_i^{(1)}=\begin{cases}
    s_i^{(1)}+v, & tiempo < lag\\
    s_i^{(1)}+s_i^{(2)}+v, & e.o.c.
    \end{cases},
\end{equation}

donde ahora $s_i^{(1)}$ corresponde al primer estímulo y $s_i^{(2)}$ corresponde al segundo estímulo.


In [6]:
def MaxSelector3capas_2_stim(stimulus,stimulus2, lag, time, delta):
    n  = np.size(stimulus)
    v  = np.zeros((n,1)) 
    v0 = v
    for t in np.arange(0.0, time, delta):
        if t<lag:
            u1 = stimulus + v
        else:
            u1 = stimulus + stimulus2 + v
        u2 = f3(u1)
        v  = v+f4(u2)
        
    return f1(u2).T

Probamos este nuevo MaxSelector con dos estímulos, primero con un estímulo nulo y un valor máximo en la cuarta entrada.

In [7]:
stimulus  = np.array([[0.0, 0.1, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).T
stimulus2 = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).T
lag       = 9.0
MaxSelector3capas_2_stim(stimulus,stimulus2, lag, time, delta)

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

Ahora agregamos un estímulo extra en la segunda entrada, tal que la suma de ambos estímulos es máxima en la segunda entrada.

In [8]:
stimulus2 = np.array([[0.0, 0.51, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).T
lag       = 9.0
MaxSelector3capas_2_stim(stimulus,stimulus2, lag, time, delta)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

Con este ejemplo podemos ver que se rompe el efecto de histéresis.